In [1]:
import pandas as pd

C:\Users\Manos\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
df = pd.read_parquet('yellow_tripdata_2022-01.parquet')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,None,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN
2463927,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,None,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN
2463928,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,None,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN
2463929,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,None,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN


In [20]:
parquet_df=df.head(100000).copy()

In [5]:
taxi_zone_df

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NV,NaN


In [21]:
# Calculate the maximum dropoff time
max_dropoff_time = df['tpep_dropoff_datetime'].max()

# Filter rows with the maximum dropoff time
latest_dropoff_df = df[df['tpep_dropoff_datetime'] == max_dropoff_time]

# Merge with taxi_zone DataFrame
taxi_zone_df = pd.read_csv("taxi_zone.csv")  # Load taxi_zone data if not available
result_df = pd.merge(latest_dropoff_df, taxi_zone_df, how='inner', left_on='DOLocationID', right_on='LocationID')
result_df = result_df[['Zone', 'tpep_dropoff_datetime']]
result_df

,Zone,tpep_dropoff_datetime
0,Lincoln Square East,2022-05-18 20:47:45


In [22]:
merged_df = pd.merge(parquet_df, taxi_zone_df, left_on='PULocationID', right_on='LocationID', how='inner')
merged_df = pd.merge(merged_df, taxi_zone_df, left_on='DOLocationID', right_on='LocationID', how='inner')

# Compute trip time between pickup and dropoff (assuming 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' are columns representing pickup and dropoff times)
merged_df['trip_time'] = merged_df['tpep_dropoff_datetime'] - merged_df['tpep_pickup_datetime']

# Rename the columns for clarity after merging
merged_df.rename(columns={'Zone_x': 'pickup_zone', 'Zone_y': 'dropoff_zone'}, inplace=True)

# Group by pickup and dropoff zones, and compute average, minimum, and maximum trip times
trip_time_stats = merged_df.groupby(['pickup_zone', 'dropoff_zone'])['trip_time'].agg(['mean', 'min', 'max']).reset_index()

# Filter out pairs where pickup_zone equals dropoff_zone
trip_time_stats = trip_time_stats[trip_time_stats['pickup_zone'] != trip_time_stats['dropoff_zone']]


In [23]:
merged_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,airport_fee,LocationID_x,Borough_x,pickup_zone,service_zone_x,LocationID_y,Borough_y,dropoff_zone,service_zone_y,trip_time
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,...,0.00,142,Manhattan,Lincoln Square East,Yellow Zone,236,Manhattan,Upper East Side North,Yellow Zone,0 days 00:17:49
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,...,0.00,236,Manhattan,Upper East Side North,Yellow Zone,42,Manhattan,Central Harlem North,Boro Zone,0 days 00:08:24
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,...,0.00,166,Manhattan,Morningside Heights,Boro Zone,166,Manhattan,Morningside Heights,Boro Zone,0 days 00:08:58
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,...,0.00,114,Manhattan,Greenwich Village South,Yellow Zone,68,Manhattan,East Chelsea,Yellow Zone,0 days 00:10:02
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,...,0.00,68,Manhattan,East Chelsea,Yellow Zone,163,Manhattan,Midtown North,Yellow Zone,0 days 00:37:32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1,2022-01-02 17:48:16,2022-01-02 18:04:03,1.0,3.10,1.0,N,186,236,1,...,0.00,186,Manhattan,Penn Station/Madison Sq West,Yellow Zone,236,Manhattan,Upper East Side North,Yellow Zone,0 days 00:15:47
99996,2,2022-01-02 17:41:15,2022-01-02 18:15:20,2.0,17.68,2.0,N,132,164,1,...,1.25,132,Queens,JFK Airport,Airports,164,Manhattan,Midtown South,Yellow Zone,0 days 00:34:05
99997,2,2022-01-02 17:01:35,2022-01-02 17:10:33,1.0,1.66,1.0,N,48,239,1,...,0.00,48,Manhattan,Clinton East,Yellow Zone,239,Manhattan,Upper West Side South,Yellow Zone,0 days 00:08:58
99998,2,2022-01-02 17:13:10,2022-01-02 17:17:02,1.0,0.95,1.0,N,143,239,1,...,0.00,143,Manhattan,Lincoln Square West,Yellow Zone,239,Manhattan,Upper West Side South,Yellow Zone,0 days 00:03:52


In [24]:
trip_time_stats.sort_values(by='mean',ascending=False)

,pickup_zone,dropoff_zone,mean,min,max
7197,Yorkville East,Steinway,0 days 23:59:33,0 days 23:59:33,0 days 23:59:33
5535,Stuy Town/Peter Cooper Village,Murray Hill-Queens,0 days 23:58:44,0 days 23:58:44,0 days 23:58:44
6672,Washington Heights North,Highbridge Park,0 days 23:58:40,0 days 23:58:40,0 days 23:58:40
5965,Two Bridges/Seward Park,Bushwick South,0 days 23:58:14,0 days 23:58:14,0 days 23:58:14
802,Clinton East,Prospect-Lefferts Gardens,0 days 23:53:56,0 days 23:53:56,0 days 23:53:56
...,...,...,...,...,...
5328,Seaport,NV,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00
2899,Jamaica,NV,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00
1673,East Williamsburg,NV,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00
250,Bedford Park,NV,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00


In [26]:
merged_df[(merged_df['pickup_zone']=='Yorkville East') & (merged_df['dropoff_zone']=='Steinway')]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,airport_fee,LocationID_x,Borough_x,pickup_zone,service_zone_x,LocationID_y,Borough_y,dropoff_zone,service_zone_y,trip_time
10691,2,2022-01-01 02:10:25,2022-01-02 02:09:58,1.0,5.41,1.0,N,262,223,1,...,0.0,262,Manhattan,Yorkville East,Yellow Zone,223,Queens,Steinway,Boro Zone,0 days 23:59:33


In [25]:
# Find the pair of taxi zones with the highest average trip time
highest_avg_trip_time_row = trip_time_stats.loc[trip_time_stats['mean'].idxmax()]

# Extract pickup and dropoff zones from the row
pickup_zone_highest_avg = highest_avg_trip_time_row['pickup_zone']
dropoff_zone_highest_avg = highest_avg_trip_time_row['dropoff_zone']

# Filter the merged DataFrame to include only trips between those pickup and dropoff zones
trips_highest_avg = merged_df[(merged_df['pickup_zone'] == pickup_zone_highest_avg) & (merged_df['dropoff_zone'] == dropoff_zone_highest_avg)]

# Count the number of trips for the pair of taxi zones with the highest average trip time
num_trips_highest_avg = len(trips_highest_avg)

print(f"Number of trips for the pair of taxi zones with the highest average trip time ({pickup_zone_highest_avg} to {dropoff_zone_highest_avg}): {num_trips_highest_avg}")


Number of trips for the pair of taxi zones with the highest average trip time (Yorkville East to Steinway): 1


In [27]:
from datetime import datetime, timedelta

# Determine the latest pickup time
latest_pickup_time = merged_df['tpep_pickup_datetime'].max()

# Calculate the start time by subtracting 17 hours from the latest pickup time
start_time = latest_pickup_time - timedelta(hours=17)

# Filter the DataFrame to include only pickups between the start time and the latest pickup time
filtered_df = merged_df[(merged_df['tpep_pickup_datetime'] >= start_time) & (merged_df['tpep_pickup_datetime'] <= latest_pickup_time)]

# Group the filtered DataFrame by pickup zones and count the number of pickups for each zone
pickup_counts = filtered_df['pickup_zone'].value_counts()

# Select the top 3 busiest zones
top_3_busiest_zones = pickup_counts.head(3)

print("Top 3 busiest zones in terms of number of pickups:")
print(top_3_busiest_zones)


Top 3 busiest zones in terms of number of pickups:
pickup_zone
LaGuardia Airport      19
Lincoln Square East    17
JFK Airport            17
Name: count, dtype: int64
